# Key Questions:

- The sales team needs to know what the busiest days of the week and hours of the day are (i.e., the days and times with the most orders) in order to schedule ads at times when there are fewer orders.
- They also want to know whether there are particular times of the day when people spend the most money, as this might inform the type of products they advertise at these times.
- Instacart has a lot of products with different price tags. Marketing and sales want to use simpler price range groupings to help direct their efforts.
- Are there certain types of products that are more popular than others? The marketing and sales teams want to know which departments have the highest frequency of product orders.

The marketing and sales teams are particularly interested in the different types of customers in their system and how their ordering behaviors differ. For example:
 - What’s the distribution among users in regards to their brand loyalty (i.e., how often do they return to Instacart)?
 - Are there differences in ordering habits based on a customer’s loyalty status?
 - Are there differences in ordering habits based on a customer’s region?
 - Is there a connection between age and family status in terms of ordering habits?
 - What different classifications does the demographic information suggest?
 - Age? Income? Certain types of goods? Family status?
 - What differences can you find in ordering habits of different customer profiles? Consider the price of orders, the frequency of orders, the products customers are ordering, and anything else you can think of.

In [55]:
# importing libraries
import pandas as pd
import numpy as np

In [56]:
# importing merged_df
merged_df = pd.read_pickle('/Volumes/external_drive/Instacart/Data/Prepared Data/instacart_merged.pkl')
merged_df.shape

(32404859, 20)

## 1. Determining the two busiest and slowest days of the week

In [57]:
# checking frequency of orders
# 0 = Saturday
# 1 = Sunday
# 2 = Monday
# 3 = Tuesday
# 4 = Wednesday
# 5 = Thursday
# 6 = Friday

merged_df.value_counts('order_dow')

order_dow
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
dtype: int64

In [58]:
# creating a new column called busiest days and adding values to column
busiest_days = []

for value in merged_df["order_dow"]:
  if value == 0 or value == 1:
    busiest_days.append("busiest days")
  elif value == 4 or value == 3:
    busiest_days.append("slowest days")
  else:
    busiest_days.append("regular day")
    
merged_df['busiest_days'] = busiest_days

In [59]:
merged_df.value_counts('busiest_days')

busiest_days
regular day     12916111
busiest days    11864412
slowest days     7624336
dtype: int64

## 2. Determining the busiest times of day

In [60]:
# checking busiest hours when orders are placed
merged_df.value_counts('order_hour_of_day')

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
dtype: int64

Most orders: 9am-4pm, Average orders: 5pm-11pm, Least orders: 12am-8am

In [61]:
# creating a new column called busiest_period_of_day and adding values to column

busiest_period = []

for hour in merged_df["order_hour_of_day"]:
  if hour in [9, 10, 11, 12, 13, 14, 15, 16]:
    busiest_period.append("most orders")
  elif hour in [17, 18, 19, 20, 21, 22, 23]:
    busiest_period.append("average orders")
  else:
    busiest_period.append("fewest orders")
merged_df['busiest_period_of_day'] = busiest_period

In [105]:
merged_df['busiest_period_of_day'].value_counts(dropna = False)

most orders       21118071
average orders     7790795
fewest orders      3495993
Name: busiest_period_of_day, dtype: int64

## 3. Adding simpler price groupings

- If the price of a product is less than or equal to 5 it will be labeled as a "low-range product".
- If the price of a product is greater than 5 but less than 15 it will be labeled as a "mid-range product".
- If the price of a product greater that 15 it will be labeled as a "high-range product".

In [63]:
# creating a new column called price_range and adding values to column

price_range = []

for prices in merged_df["prices"]:
  if prices <= 5:
    price_range.append("low-range product")
  elif prices > 5 and prices <=15:
    price_range.append("mid-range product")
  else:
    price_range.append("high-range product")
merged_df['price_range'] = price_range

In [104]:
merged_df['price_range'].value_counts(dropna = False)

mid-range product     21865987
low-range product     10126321
high-range product      412551
Name: price_range, dtype: int64

# 4. Grouping users by loyalty status

- If the maximum orders the user has made is over 40, then the customer will be labeled a “loyal customer.”
- If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled a “regular customer.”
- If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled a “new customer.”

In [65]:
# creating a column showing the maximum number of orders per customer
merged_df['max_order'] = merged_df.groupby(['user_id'])['order_number'].transform(np.max)

In [66]:
# creating loyalty_flag column and adding 'loyal customers'
merged_df.loc[merged_df['max_order'] > 40, 'loyalty_flag'] = 'loyal customer'

In [67]:
# adding 'regular customers' to column
merged_df.loc[(merged_df['max_order'] <= 40) & (merged_df['max_order'] > 10), 'loyalty_flag'] = 'regular customer'

In [68]:
# adding 'new customers' to column
merged_df.loc[merged_df['max_order'] < 40, 'loyalty_flag'] = 'new customer'

In [103]:
merged_df['loyalty_flag'].value_counts(dropna = False)

new customer        21753455
loyal customer      10284093
regular customer      367311
Name: loyalty_flag, dtype: int64

# 5. Grouping users by average spend 

- If the mean of the prices of products purchased by a user is lower than 10, they are flagged as a “low spender.”
- If the mean of the prices of products purchased by a user is higher than or equal to 10, they are flagged as a “high spender.”

In [70]:
# creating a column showing the average price of products purchased per customer
merged_df['avg_order'] = merged_df.groupby(['user_id'])['prices'].transform(np.mean).round(2)

In [71]:
# creating spending_flag column and adding 'high spenders'
merged_df.loc[merged_df['avg_order'] >= 10, 'spending_flag'] = 'high spender'

In [72]:
# adding 'ligh spenders' to column
merged_df.loc[merged_df['avg_order'] < 10, 'spending_flag'] = 'low spender'

In [102]:
merged_df['spending_flag'].value_counts(dropna = False)

low spender     32284249
high spender      120610
Name: spending_flag, dtype: int64

# 6. Grouping users by frequency of orders

- If the median of “days_since_prior_order” is higher than 20, then the customer will be labeled a “non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer will be labeled a “regular customer.”
- If the median is lower than or equal to 10, then the customer will be labeled a “frequent customer.”

In [74]:
# adding median_days_since_prior_order column to df
merged_df['median_days_prior_order'] = merged_df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [75]:
# creating frequency_flag column and adding 'non-frequent customers'
merged_df.loc[merged_df['median_days_prior_order'] >20, 'frequency_flag'] = 'non-frequent customer'

In [76]:
# adding 'regular customers' to column
merged_df.loc[(merged_df['median_days_prior_order'] >10) & (merged_df['median_days_prior_order'] <= 20), 'frequency_flag'] = 'regular customer'

In [77]:
# adding 'frequent customers' to column
merged_df.loc[merged_df['median_days_prior_order'] <=10, 'frequency_flag'] = 'frequent customer'

In [109]:
merged_df['frequency_flag'].value_counts(dropna = False)

frequent customer        21559853
regular customer          7208564
non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [108]:
# investigating NaN values identified above
nan_values = merged_df[merged_df['median_days_prior_order'].isna()]
nan_values

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,product_name,aisle_id,department_id,...,busiest_period_of_day,price_range,max_order,loyalty_flag,avg_order,spending_flag,median_days_prior_order,frequency_flag,region,age_group
31821430,895835,159838,1,0,17,NaN,33401,Goat Cheese Crumbles,21,16,...,average orders,mid-range product,1,new customer,7.42,low spender,NaN,NaN,South,55-64
31821431,895835,159838,1,0,17,NaN,22198,4X Ultra Concentrated Natural Laundry Detergen...,75,17,...,average orders,low-range product,1,new customer,7.42,low spender,NaN,NaN,South,55-64
31821432,895835,159838,1,0,17,NaN,21334,Organic Peeled Garlic,123,4,...,average orders,mid-range product,1,new customer,7.42,low spender,NaN,NaN,South,55-64
31821433,895835,159838,1,0,17,NaN,10749,Organic Red Bell Pepper,83,4,...,average orders,mid-range product,1,new customer,7.42,low spender,NaN,NaN,South,55-64
31821434,895835,159838,1,0,17,NaN,23695,California Veggie Burger,42,1,...,average orders,low-range product,1,new customer,7.42,low spender,NaN,NaN,South,55-64


In [147]:
# the NaN values above are as a result of the customer only having placed 1 order.
# these customers don't contribute significantly to revenue or this analysis and will thus be removed.
merged_df.dropna(subset=['frequency_flag'], inplace=True)

In [148]:
merged_df['frequency_flag'].value_counts(dropna = False)

frequent customer        21559853
regular customer          7208564
non-frequent customer     3636437
Name: frequency_flag, dtype: int64

# 7. Segmenting users based on region

In [113]:
# creating regional classifiers in df based on regional divisions used by the United States Census Bureau
merged_df.loc[merged_df['state'].isin(['Wisconsin',
                         'Michigan',
                         'Illinois',
                         'Indiana',
                         'Ohio',
                         'North Dakota',
                         'South Dakota',
                         'Nebraska',
                         'Kansas',
                         'Minnesota',
                         'Iowa',
                         'Missouri']), 'region'] = 'Midwest'

In [114]:
merged_df.loc[merged_df['state'].isin(['Maine', 
                         'New Hampshire', 
                         'Vermont',
                         'Massachusetts',
                         'Rhode Island',
                         'Connecticut',
                         'New York',
                         'Pennsylvania',
                         'New Jersey']), 'region'] = 'Northeast'

In [115]:
merged_df.loc[merged_df['state'].isin(['Delaware', 
                         'Maryland', 
                         'District of Columbia',
                         'Virginia',
                         'West Virginia',
                         'North Carolina',
                         'South Carolina',
                         'Georgia',
                         'Florida',
                         'Kentucky',
                         'Tennessee',
                         'Mississippi',
                         'Alabama',
                         'Oklahoma',
                         'Texas',
                         'Arkansas',
                         'Louisiana']), 'region'] = 'South'

In [116]:
merged_df.loc[merged_df['state'].isin(['Idaho', 
                         'Montana', 
                         'Wyoming',
                         'Nevada',
                         'Utah',
                         'Colorado',
                         'Arizona',
                         'New Mexico',
                         'Alaska',
                         'Washington',
                         'Oregon',
                         'California',
                         'Hawaii']), 'region'] = 'West'

In [117]:
merged_df['region'].value_counts(dropna = False)

South        10791880
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

# 8. Segmenting customers based on age

In [118]:
# creating age groups for segmentation, we will use the same segments that are available for Google Ads: 
# '18–24', '25–34', '35–44', '45–54', '55–64', '65+'

In [119]:
merged_df.loc[(merged_df['age'] >= 18) & (merged_df['age'] <=24), 
'age_group'] = '18-24' 

In [120]:
merged_df.loc[(merged_df['age'] >= 25) & (merged_df['age'] <=34), 
'age_group'] = '25-34'

In [121]:
merged_df.loc[(merged_df['age'] >= 35) & (merged_df['age'] <=44), 
'age_group'] = '35-44'

In [122]:
merged_df.loc[(merged_df['age'] >= 45) & (merged_df['age'] <=54), 
'age_group'] = '45-54'

In [123]:
merged_df.loc[(merged_df['age'] >= 55) & (merged_df['age'] <=64), 
'age_group'] = '55-64'

In [124]:
merged_df.loc[merged_df['age'] >= 65, 'age_group'] = '65+'

In [125]:
merged_df['age_group'].value_counts(dropna = False)

65+      8575084
45-54    5111126
35-44    5062429
25-34    5054654
55-64    5028560
18-24    3573001
Name: age_group, dtype: int64

# 8. Segmenting customers based on income

In [126]:
# creating income tiers for segmentation, we will use the same segments that are defined by the Pew Research Center
# $48,500 - $145,500 = Middle-income
# $145,500+ = Upper-income
# <$48,500 = Lower_income

In [127]:
merged_df.loc[(merged_df['income'] >= 48500) & (merged_df['income'] <=145500), 
'income_tier'] = 'Middle_income' 

In [128]:
merged_df.loc[merged_df['income'] >145500, 'income_tier'] = 'Upper-income'

In [129]:
merged_df.loc[merged_df['income'] <48500, 'income_tier'] = 'Lower-income'

In [131]:
merged_df['income_tier'].value_counts(dropna = False)

Middle_income    24158818
Upper-income      4881048
Lower-income      3364988
Name: income_tier, dtype: int64

# 9. Segmenting customers based on family status

In [132]:
# creating family_profile for segmentation
#'Married, no children', 'Married, children', 'Single, children', 'Single, no children'

In [139]:
merged_df.loc[(merged_df['fam_status'] == 'married') & (merged_df['n_dependants'] == 1), 'family_profile'] = 'Married, no children'

In [141]:
merged_df.loc[(merged_df['fam_status'] == 'married') & (merged_df['n_dependants'] >= 2), 'family_profile'] = 'Married, children'

In [143]:
merged_df.loc[(merged_df['fam_status'] != 'married') & (merged_df['n_dependants'] == 0), 'family_profile'] = 'Single, no children'

In [145]:
merged_df.loc[(merged_df['fam_status'] != 'married') & (merged_df['n_dependants'] >= 1), 'family_profile'] = 'Single, children'

In [146]:
merged_df['family_profile'].value_counts(dropna = False)

Married, children       15205844
Single, no children      8097503
Married, no children     7550904
Single, children         1550603
Name: family_profile, dtype: int64

# 10. Exporting dataset

In [151]:
merged_df.to_pickle('/Volumes/external_drive/Instacart/Data/Prepared Data/final_df.pkl')